In [1]:
import pandas as pd
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

## Importamos los archivos

In [2]:
df_bajas = pd.read_feather('../../../data/processed/periodos_baja_x_cif_id.feather')

In [3]:
df_pagos = pd.read_feather('../../../data/processed/pagos_x_cif_id.feather')

In [4]:
df_interacciones = pd.read_feather('../../../data/processed/interacciones_tipo_periodo_x_cif_id.feather')

In [5]:
df_clientes = pd.read_feather('../../../data/interim/clientes.feather')

In [6]:
df_clientes = df_clientes.rename(columns={"ID":"CIF_ID"})

In [7]:
df_siniestros = pd.read_feather('../../../data/processed/siniestros_x_cif_id.feather')

## Verificamos que todos se puedan combinar con la tabla Bajas

In [8]:
def check_merge(df1,df2,field):
    return len(set(df1[field].unique()).intersection(set(df2[field].unique()))) / len(df1[field].unique())

In [9]:
check_merge(df_bajas, df_pagos, "CIF_ID")

0.9920195773212915

In [10]:
check_merge(df_bajas, df_clientes, "CIF_ID")

0.9868415534564975

*Renombrando columnas de interacciones y descartando periodos que no vamos a analizar*

In [11]:
df_interacciones.columns

Index(['CIF_ID', '1_TIPOINT_A', '1_TIPOINT_I', '1_TIPOINT_O', '2_TIPOINT_A',
       '2_TIPOINT_I', '2_TIPOINT_O', '3_TIPOINT_A', '3_TIPOINT_I',
       '3_TIPOINT_O', '4_TIPOINT_A', '4_TIPOINT_I', '4_TIPOINT_O',
       '5_TIPOINT_A', '5_TIPOINT_I', '5_TIPOINT_O', '6_TIPOINT_A',
       '6_TIPOINT_I', '6_TIPOINT_O', '7_TIPOINT_A', '7_TIPOINT_I',
       '7_TIPOINT_O', '8_TIPOINT_A', '8_TIPOINT_I', '8_TIPOINT_O',
       '9_TIPOINT_A', '9_TIPOINT_I', '9_TIPOINT_O', '10_TIPOINT_A',
       '10_TIPOINT_I', '10_TIPOINT_O', '11_TIPOINT_A', '11_TIPOINT_I',
       '11_TIPOINT_O', '12_TIPOINT_A', '12_TIPOINT_I', '12_TIPOINT_O',
       '13_TIPOINT_A', '13_TIPOINT_I', '13_TIPOINT_O', '14_TIPOINT_A',
       '14_TIPOINT_I', '14_TIPOINT_O', '15_TIPOINT_A', '15_TIPOINT_I',
       '15_TIPOINT_O', '16_TIPOINT_A', '16_TIPOINT_I', '16_TIPOINT_O',
       '17_TIPOINT_A', '17_TIPOINT_I', '17_TIPOINT_O', '18_TIPOINT_A',
       '18_TIPOINT_I', '18_TIPOINT_O'],
      dtype='object')

In [12]:
df_interacciones = df_interacciones.rename(columns=lambda x: x.replace("(","").replace(")","").replace(", ","_TIPOINT_").replace("'","")) 

In [12]:
to_drop = [col for col in df_interacciones.drop('CIF_ID', axis=1).columns if int(col.split('_')[0])>18]

In [13]:
to_drop

[]

In [ ]:
df_interacciones.drop(to_drop, axis=1, inplace=True)

In [14]:
check_merge(df_bajas, df_interacciones, "CIF_ID")

0.9998001810704867

In [15]:
check_merge(df_bajas, df_siniestros, "CIF_ID")

0.09662602037166321

## Merge entre tablas

In [16]:
len(df_bajas)

405367

In [17]:
len(df_interacciones)

1872729

In [18]:
len(df_pagos)

979749

In [19]:
len(df_clientes)

11650636

In [20]:
len(df_siniestros)

92314

In [21]:
df_bajas_int =  pd.merge(df_bajas, df_interacciones, on=['CIF_ID'], how='left')

In [22]:
df_bajas_int.head()

,CIF_ID,hist_polizas,periodo_baja,1_TIPOINT_A,1_TIPOINT_I,1_TIPOINT_O,2_TIPOINT_A,2_TIPOINT_I,2_TIPOINT_O,3_TIPOINT_A,3_TIPOINT_I,3_TIPOINT_O,4_TIPOINT_A,4_TIPOINT_I,4_TIPOINT_O,5_TIPOINT_A,5_TIPOINT_I,5_TIPOINT_O,6_TIPOINT_A,6_TIPOINT_I,6_TIPOINT_O,7_TIPOINT_A,7_TIPOINT_I,7_TIPOINT_O,8_TIPOINT_A,8_TIPOINT_I,8_TIPOINT_O,9_TIPOINT_A,9_TIPOINT_I,9_TIPOINT_O,10_TIPOINT_A,10_TIPOINT_I,10_TIPOINT_O,11_TIPOINT_A,11_TIPOINT_I,11_TIPOINT_O,12_TIPOINT_A,12_TIPOINT_I,12_TIPOINT_O,13_TIPOINT_A,13_TIPOINT_I,13_TIPOINT_O,14_TIPOINT_A,14_TIPOINT_I,14_TIPOINT_O,15_TIPOINT_A,15_TIPOINT_I,15_TIPOINT_O,16_TIPOINT_A,16_TIPOINT_I,16_TIPOINT_O,17_TIPOINT_A,17_TIPOINT_I,17_TIPOINT_O,18_TIPOINT_A,18_TIPOINT_I,18_TIPOINT_O
0,1313.0,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,-1,NaN,1.0,2.0,NaN,NaN,2.0,NaN,4.0,2.0,NaN,1.0,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,1.0,3.0,NaN,NaN,2.0,NaN,NaN,4.0,NaN,NaN,3.0,NaN,NaN,2.0,NaN,NaN,5.0,NaN,NaN,2.0,NaN,NaN,6.0,NaN,NaN,2.0,NaN,1.0,2.0
1,1482.0,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,-1,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN,4.0,NaN,NaN,3.0,NaN,NaN,5.0,NaN,NaN,4.0,NaN,NaN,3.0,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,3.0,NaN,NaN,4.0,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,4.0,NaN,NaN,3.0
2,1559.0,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,-1,NaN,8.0,1.0,NaN,4.0,3.0,NaN,6.0,3.0,NaN,5.0,3.0,NaN,2.0,1.0,NaN,3.0,4.0,NaN,5.0,6.0,NaN,1.0,3.0,NaN,2.0,2.0,1.0,9.0,6.0,NaN,NaN,4.0,NaN,NaN,3.0,NaN,NaN,2.0,NaN,NaN,4.0,NaN,NaN,4.0,NaN,1.0,8.0,NaN,NaN,2.0,NaN,4.0,9.0
3,1779.0,0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,13.0,10.0,NaN,NaN,3.0,NaN,NaN,1.0,NaN,NaN,4.0,NaN,NaN,2.0,NaN,NaN,1.0
4,2994.0,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,3.0,NaN,1.0,1.0,NaN,5.0,7.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,2.0,NaN,1.0,1.0,NaN,NaN,2.0,NaN,1.0,NaN,NaN,1.0,2.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,NaN


In [23]:
df_bajas_int_sini =  pd.merge(df_bajas_int, df_siniestros, on=['CIF_ID'], how='left')

In [24]:
df_bajas_int_sini = df_bajas_int_sini.fillna(0)

In [25]:
df_bajas_int_sini.head()

,CIF_ID,hist_polizas,periodo_baja,1_TIPOINT_A,1_TIPOINT_I,1_TIPOINT_O,2_TIPOINT_A,2_TIPOINT_I,2_TIPOINT_O,3_TIPOINT_A,3_TIPOINT_I,3_TIPOINT_O,4_TIPOINT_A,4_TIPOINT_I,4_TIPOINT_O,5_TIPOINT_A,5_TIPOINT_I,5_TIPOINT_O,6_TIPOINT_A,6_TIPOINT_I,6_TIPOINT_O,7_TIPOINT_A,7_TIPOINT_I,7_TIPOINT_O,8_TIPOINT_A,8_TIPOINT_I,8_TIPOINT_O,9_TIPOINT_A,9_TIPOINT_I,9_TIPOINT_O,10_TIPOINT_A,10_TIPOINT_I,10_TIPOINT_O,11_TIPOINT_A,11_TIPOINT_I,11_TIPOINT_O,12_TIPOINT_A,12_TIPOINT_I,12_TIPOINT_O,13_TIPOINT_A,13_TIPOINT_I,13_TIPOINT_O,14_TIPOINT_A,14_TIPOINT_I,14_TIPOINT_O,15_TIPOINT_A,15_TIPOINT_I,15_TIPOINT_O,16_TIPOINT_A,16_TIPOINT_I,16_TIPOINT_O,17_TIPOINT_A,17_TIPOINT_I,17_TIPOINT_O,18_TIPOINT_A,18_TIPOINT_I,18_TIPOINT_O,periodo_sini_1,periodo_sini_2,periodo_sini_3,periodo_sini_4,periodo_sini_5,periodo_sini_6,periodo_sini_7,periodo_sini_8,periodo_sini_9,periodo_sini_10,periodo_sini_11,periodo_sini_12,periodo_sini_13,periodo_sini_14,periodo_sini_15,periodo_sini_16,periodo_liquidacion_sini_1,periodo_liquidacion_sini_2,periodo_liquidacion_sini_3,periodo_liquidacion_sini_4,periodo_liquidacion_sini_5,periodo_liquidacion_sini_6,periodo_liquidacion_sini_7,periodo_liquidacion_sini_8,periodo_liquidacion_sini_9,periodo_liquidacion_sini_10,periodo_liquidacion_sini_11,periodo_liquidacion_sini_12,periodo_liquidacion_sini_13,periodo_liquidacion_sini_14,periodo_liquidacion_sini_15,periodo_liquidacion_sini_16,periodo_rechazo_sini_1,periodo_rechazo_sini_2,periodo_rechazo_sini_3,periodo_rechazo_sini_4,periodo_rechazo_sini_5,periodo_rechazo_sini_6,periodo_rechazo_sini_7,periodo_rechazo_sini_8,periodo_rechazo_sini_9,periodo_rechazo_sini_10,periodo_rechazo_sini_11,periodo_rechazo_sini_12,periodo_rechazo_sini_13,periodo_rechazo_sini_14,periodo_rechazo_sini_15,periodo_rechazo_sini_16
0,1313.0,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,-1,0.0,1.0,2.0,0.0,0.0,2.0,0.0,4.0,2.0,0.0,1.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,3.0,0.0,0.0,2.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0,6.0,0.0,0.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1482.0,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,-1,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1559.0,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,-1,0.0,8.0,1.0,0.0,4.0,3.0,0.0,6.0,3.0,0.0,5.0,3.0,0.0,2.0,1.0,0.0,3.0,4.0,0.0,5.0,6.0,0.0,1.0,3.0,0.0,2.0,2.0,1.0,9.0,6.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,1.0,8.0,0.0,0.0,2.0,0.0,4.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1779.0,0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,10.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2994.0,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.0,0.0,1.0,1.0,0.0,5.0,7.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [26]:
from functools import reduce
df_merged = reduce(lambda  left,right: pd.merge(left, right, on=['CIF_ID'], how='inner'), [df_bajas_int_sini, df_pagos, df_clientes])

In [27]:
df_merged.shape

(396835, 308)

### Guardamos la tabla completa

In [28]:
df_merged.to_feather('../../../data/processed/merged_data_18_periods.feather')

--- 

In [1]:
import pandas as pd
df_merged = pd.read_feather('../../../data/processed/merged_data_18_periods.feather')

In [ ]:
"], [".join(set([''.join([s for s in col if not s.isdigit()]) for col in df_merged.columns]))